In [ ]:
%matplotlib inline

import numpy as np
import menpo.io as mio
from menpo.image import Image
from menpo.feature import ndfeature, greyscale, no_op, igo, centralize
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images
from alaborticcv2015.deepconvkernel.discriminative import DiscriminativeLDCN, learn_svm_filters

In [ ]:
@ndfeature
def grey_igo(pixels):
    pixels = greyscale(pixels)
    pixels = igo (pixels)
    return pixels

def greyscale_average(pixels):
    return greyscale(pixels, mode='average')

# SVM Net 

Load data:

In [ ]:
folder_path = '/Users/joan/PhD/alaborticcv2015/'

In [ ]:
images = []
for i in mio.import_images('/Users/joan/PhD/DataBases/faces/lfpw/**/*', verbose=True, 
                           max_images=100):
    i.crop_to_landmarks_proportion_inplace(0.5)
    i = i.rescale_landmarks_to_diagonal_range(100)
    labeller(i, 'PTS', ibug_face_66)
    images.append(i)

In [ ]:
visualize_images(images)

Set network options:

In [ ]:
# general network options
features = greyscale_average
n_layers = 2
architecture = 1
normalize_patches = centralize
normalize_filters = None

# feature images
images = [features(i) for i in images] 

# class labels
class_labels = ['jaw', 
                'left_eyebrow', 
                'right_eyebrow', 
                'nose', 'left_eye', 
                'right_eye', 
                'mouth']

# number of filters and patch sizes
patch_shape = (5, 5)

Build a network:

In [ ]:
net = DiscriminativeLDCN(learn_filters=learn_svm_filters, 
                         n_layers=n_layers,
                         architecture=architecture,
                         normalize_patches=normalize_patches,
                         normalize_filters=normalize_filters,
                         patch_shape=patch_shape)

net.learn_network_from_class_labels(images, 
                                    class_labels, 
                                    group='ibug_face_66', 
                                    verbose=True)

In [ ]:
print '# of layers:', net.n_layers
print '# of filters:', net.n_filters
print '# of filters per layer:', net.n_filters_layer
print 'filters shape:', net.filters_shape

Save network:

In [ ]:
# if save_net:
#     file_name = (folder_path
#                  + 'networks/svm_net/svm_net_'
#                  + features.__name__ + '_'
#                  + str(1) + '_' 
#                  + str(architecture) + '_'
#                  + str(patch_shape) + '.pkl.gz') 
#     mio.export_pickle(net, file_name, overwrite=True)

Visualize the previous filters using the previous LinDeepConvNet:

In [ ]:
filters = net.filters_spatial()

First layer spatial filters:

In [ ]:
visualize_images(filters[0])

Second layer spatial filters:

In [ ]:
visualize_images(filters[1])

We can also visualize the previous filters on the frequency domain:

In [ ]:
filters_frequecy = net.filters_frequency(ext_shape=(100, 100))

First layer filters in the frequency domain:

In [ ]:
visualize_images(filters_frequecy[0])

Second layer filters in the frequency domain:

In [ ]:
visualize_images(filters_frequecy[1])

Visualize Deep Convolutional Kernels:

In [ ]:
kernels = net.kernels_spatial()

In [ ]:
visualize_images(kernels)

In [ ]:
kernels = net.kernels_frequency(ext_shape=(100, 100))

In [ ]:
visualize_images(kernels)

Load images:

In [ ]:
test_images = []
for i in mio.import_images('/Users/joan/PhD/DataBases/faces/afw/', verbose=True, 
                           max_images=10):
    i.crop_to_landmarks_proportion_inplace(1)
    i = i.rescale_landmarks_to_diagonal_range(100)
    i = features(i)
    test_images.append(i)

Compute network features:

In [ ]:
network_responses = [net.network_response(i) for i in test_images]

In [ ]:
visualize_images(network_responses)

Compute kernel features:

In [ ]:
kernel_responses = [net.kernel_response(i) for i in test_images]

In [ ]:
visualize_images(network_responses)

Check network and kernel equivalence:

In [ ]:
for (nr, kr, i) in zip(network_responses, kernel_responses, test_images):
    print 'Network response dot product:',  nr.as_vector().dot(nr.as_vector())
    print 'Kernel response dot product:', kr.as_vector().dot(kr.as_vector())
    
    fft_i = np.fft.fft2(i.pixels) / np.sqrt(np.prod(i.shape[-2:]))
    print 'Kernel Fourier dot product:', np.sum(net._compute_kernel(ext_shape=fft_i.shape[-2:]) * 
                                                fft_i.conj() * fft_i)